In [23]:
import json

# Read JSON data from file
with open("json-fixer.json", "r") as file:
    data = json.load(file)

# Iterate through the data
for fold_data in data:
    fold = fold_data['fold']
    source = fold_data['source']
    document_id = fold_data['documentId']
    document_text = fold_data['documentText']

    # Iterate through the passages
    for passage in fold_data['passages']:
        passage_id = passage['passageId']
        passage_start = passage['passageStart']
        passage_end = passage['passageEnd']
        passage_text = passage['passageText'].replace("'", "\\'")

        # Iterate through exhaustivelyAnnotatedProperties
        for property_data in passage['exhaustivelyAnnotatedProperties']:
            property_id = property_data['propertyId']
            property_name = property_data['propertyName']
            property_description = property_data['propertyDescription'].replace("'", "\\'")

            # Create Cypher query for Property
            cypher_query_property = f'''
            CREATE (p:Passage {{passageId: '{passage_id}', passageStart: {passage_start}, passageEnd: {passage_end}, passageText: '{passage_text}'}})
            CREATE (pr:Property {{propertyId: '{property_id}', propertyName: '{property_name}', propertyDescription: '{property_description}'}})
            CREATE (p)-[:HAS_PROPERTY]->(pr)
            '''

            print(cypher_query_property)

        # Iterate through facts (if not empty)
        if 'facts' in passage:
            for fact in passage['facts']:
                fact_id = fact['factId']
                subject_start = fact['subjectStart']
                subject_end = fact['subjectEnd']
                subject_text = fact['subjectText'].replace("'", "\\'")
                subject_uri = fact['subjectUri']
                object_start = fact['objectStart']
                object_end = fact['objectEnd']
                object_text = fact['objectText'].replace("'", "\\'")
                object_uri = fact['objectUri']

                # Create Cypher query for Fact
                cypher_query_fact = f'''
                CREATE (f:Fact {{factId: '{fact_id}'}})
                CREATE (s:Subject {{subjectStart: {subject_start}, subjectEnd: {subject_end}, subjectText: '{subject_text}', subjectUri: '{subject_uri}'}})
                CREATE (o:Object {{objectStart: {object_start}, objectEnd: {object_end}, objectText: '{object_text}', objectUri: '{object_uri}'}})
                CREATE (f)-[:HAS_SUBJECT]->(s)
                CREATE (f)-[:HAS_OBJECT]->(o)
                CREATE (p)-[:HAS_FACT]->(f)
                '''

                print(cypher_query_fact)



            CREATE (p:Passage {passageId: '13205:453:779', passageStart: 453, passageEnd: 779, passageText: 'When she founded Apex Legal Document Preparation Services she became a Premium Member of the FALDP (Florida Association of Legal Document Preparers) and educated herself on the different types of divorce been offered across the State of Florida and learned from the best in the industry of legal document preparation services.'})
            CREATE (pr:Property {propertyId: '9', propertyName: 'EDUCATED_AT', propertyDescription: 'Describes the relationship between a person and the institution where she/he studied.'})
            CREATE (p)-[:HAS_PROPERTY]->(pr)
            

            CREATE (p:Passage {passageId: '13115:1833:1922', passageStart: 1833, passageEnd: 1922, passageText: 'He is a member of Phi Beta Kappa and was a Fulbright Fellow in Law in Brazil (1980-1981).'})
            CREATE (pr:Property {propertyId: '10', propertyName: 'NATIONALITY', propertyDescription: 'Des

In [25]:
!pip install neo4j

  Using cached neo4j-5.7.0.tar.gz (176 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.7.0-py3-none-any.whl size=243969 sha256=95df57b5be791336c887ef50de269a569ef5514e07f2a6ebc80331efee74663e
  Stored in directory: c:\users\ravi_kumar2\appdata\local\pip\cache\wheels\6d\be\1e\749b84bba4b7429145ea09ccb46114fab4748625eb319f286a
Successfully built neo4j


In [26]:
from neo4j import GraphDatabase

In [34]:
uri = "bolt://localhost:7687/knwgrph"
# create a driver object
driver = GraphDatabase.driver(uri, auth=("neo4j", "B@bu12345"))

In [35]:
print("connected: {}".format(driver.verify_connectivity()))

connected: None


In [36]:
query_string = '''
MATCH (n)
RETURN n
'''

session = driver.session()

In [38]:
databases = list(session.run("SHOW DATABASES"))
print(databases)

[<Record name='know' type='standard' aliases=[] access='read-write' address='localhost:7687' role='primary' writer=True requestedStatus='online' currentStatus='online' statusMessage='' default=False home=False constituents=[]>, <Record name='knwgrph' type='standard' aliases=[] access='read-write' address='localhost:7687' role='primary' writer=True requestedStatus='online' currentStatus='online' statusMessage='' default=False home=False constituents=[]>, <Record name='neo4j' type='standard' aliases=[] access='read-write' address='localhost:7687' role='primary' writer=True requestedStatus='online' currentStatus='online' statusMessage='' default=True home=True constituents=[]>, <Record name='system' type='system' aliases=[] access='read-write' address='localhost:7687' role='primary' writer=True requestedStatus='online' currentStatus='online' statusMessage='' default=False home=False constituents=[]>]


In [39]:
from neo4j import GraphDatabase

# Connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "B@bu12345"
database = "knwgrph"   # database name

with open("json-fixer.json", "r") as file:
    data = json.load(file)

# Connect to the Neo4j database
driver = GraphDatabase.driver(uri, auth=(username, password), encrypted=False)
with driver.session(database=database) as session:
    # Iterate through the folds
    for fold_data in data['fold']:
        fold = fold_data['fold']
        source = fold_data['source']
        document_id = fold_data['documentId']
        document_text = fold_data['documentText']
        
        # Create document node
        session.run("CREATE (d:Document {id: $document_id, text: $document_text})", document_id=document_id, document_text=document_text)
        
        # Iterate through the passages
        for passage_data in fold_data['passages']:
            passage_id = passage_data['passageId']
            passage_start = passage_data['passageStart']
            passage_end = passage_data['passageEnd']
            passage_text = passage_data['passageText']
            
            # Create passage node
            session.run("CREATE (p:Passage {id: $passage_id, start: $passage_start, end: $passage_end, text: $passage_text})", passage_id=passage_id, passage_start=passage_start, passage_end=passage_end, passage_text=passage_text)
            
            # Iterate through the exhaustivelyAnnotatedProperties
            for property_data in passage_data['exhaustivelyAnnotatedProperties']:
                property_id = property_data['propertyId']
                property_name = property_data['propertyName']
                property_desc = property_data['propertyDescription']
                
                # Create property node
                session.run("CREATE (pr:Property {propertyId: $property_id, propertyName: $property_name, propertyDescription: $property_desc})", property_id=property_id, property_name=property_name, property_desc=property_desc)
                
            # Iterate through the facts
            for fact_data in passage_data['facts']:
                fact_id = fact_data['factId']
                subject_start = fact_data['subjectStart']
                subject_end = fact_data['subjectEnd']
                subject_text = fact_data['subjectText'].replace("'", "\\'")  # escape single quotes
                subject_uri = fact_data['subjectUri']
                object_start = fact_data['objectStart']
                object_end = fact_data['objectEnd']
                object_text = fact_data['objectText'].replace("'", "\\'")  # escape single quotes
                object_uri = fact_data['objectUri']
                property_id = fact_data['propertyId']
                human_readable = fact_data['humanReadable']
                annotated_passage = fact_data['annotatedPassage']
                
                # Create fact node
                session.run("CREATE (f:Fact {id: $fact_id, subjectStart: $subject_start, subjectEnd: $subject_end, subjectText: $subject_text, subjectUri: $subject_uri, objectStart: $object_start, objectEnd: $object_end, objectText: $object_text, objectUri: $object_uri, propertyId: $property_id, humanReadable: $human_readable, annotatedPassage: $annotated_passage})", 
                            fact_id=fact_id, subject_start=subject_start, subject_end=subject_end, subject_text=subject_text, subject_uri=subject_uri, object_start=object_start, object_end=object_end, object_text=object_text, object_uri=object_uri, property_id=property_id,


SyntaxError: unexpected EOF while parsing (20736855.py, line 61)

from neo4j import GraphDatabase
import json
import uuid  

# Connect to Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "B@bu12345"
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session(database="knwgrph")

# Load the JSON data
with open("trail.json", "r") as file:
    data = json.load(file)

# Iterate through the folds
for fold_data in data:
    fold = fold_data['fold']
    print("In Fold loop{}".format(fold))
    source = fold_data['source']
    document_id = fold_data['documentId']
    document_text = fold_data['documentText'].replace("'", "\\'")

    # Create the Document node
    query = f"CREATE (d:Document {{documentId: '{document_id}', documentText: '{document_text}'}})"
    session.run(query)

    # Iterate through the passages
    for passage_data in fold_data['passages']:
        passage_id = passage_data['passageId']
        print("In Passage loop{}".format(passage_id))
        passage_start = passage_data['passageStart']
        passage_end = passage_data['passageEnd']
        passage_text = passage_data['passageText'].replace("'", "\\'")

        # Create the Passage node
        query = f"CREATE (p:Passage {{passageId: '{passage_id}', passageStart: {passage_start}, passageEnd: {passage_end}, passageText: '{passage_text}'}})"
        session.run(query)

        # Create the Document-Passage relationship
        query = f"MATCH (d:Document {{documentId: '{document_id}'}}), (p:Passage {{passageId: '{passage_id}'}}) CREATE (d)-[:HAS_PASSAGE]->(p)"
        session.run(query)

        # Iterate through the exhaustivelyAnnotatedPropertie            
        for property_data in passage_data['exhaustivelyAnnotatedProperties']:
            property_id = property_data['propertyId']
            property_name = property_data['propertyName']
            property_description = property_data['propertyDescription'].replace("'", "\\'")

            # Create the Property node
            prop_id = uuid.uuid1() 
            query = f"CREATE (pr:Property {{prop_id: '{prop_id}',propertyId: '{property_id}', propertyName: '{property_name}', propertyDescription: '{property_description}'}})"
            session.run(query)

            
            # Create the Document-Passage relationship
            query = f"MATCH (p:Passage {{passageId: '{passage_id}'}}), (pr:Property {{prop_id: '{prop_id}'}}) CREATE (p)-[:HAS_PROPERTY]->(pr)"
            session.run(query)
                
        # Iterate through the facts
        for fact in passage_data['facts']:
            # Extract information from the fact
            fact_id = fact['factId']
            property_id = fact['propertyId']
            human_readable = fact['humanReadable']
            annotated_passage = fact['annotatedPassage']
            subject_start = fact['subjectStart']
            subject_end = fact['subjectEnd']
            subject_text = fact['subjectText'].replace("'", "\\'")
            subject_uri = fact['subjectUri']
            object_start = fact['objectStart']
            object_end = fact['objectEnd']
            object_text = fact['objectText'].replace("'", "\\'")
            object_uri = fact['objectUri']
            #property_name = fact['propertyName'].replace("'", "\\'")
            #property_desc = fact['propertyDescription'].replace("'", "\\'")
            humarn_readable = human_readable.replace("'", "\\'")
            annotated_passage = annotated_passage.replace("'", "\\'")
            
            '''
                        # Create the Property node if it doesn't exist
            property_query = f"MERGE (pr:Property {{propertyId: '{property_id}'}}) \
                            ON CREATE SET pr.propertyName = '{property_name}', \
                                            pr.propertyDescription = '{property_description}'"
            session.run(property_query)
            '''
            
            # Create the Fact node
            fact_query = f"MERGE (f:Fact {{factId: '{fact_id}'}}) \
                    ON CREATE SET f.humanReadable = '{humarn_readable}', \
                                f.annotatedPassage = '{annotated_passage}', f.propertyId = '{property_id}'"
            session.run(fact_query)
            
            query = f"MATCH (p:Passage {{passageId: '{passage_id}'}}), (f:Fact {{factId: '{fact_id}'}}) CREATE (p)-[:HAS_FACT]->(f)"
            session.run(query)
            
            # Create the Subject node if it doesn't exist
            subject_query = f"MERGE (s:Subject {{subjectUri: '{subject_uri}',start: {subject_start}, end: {subject_end}}}) \
                            ON CREATE SET s.subjectText = '{subject_text}',s.start= {subject_start}, s.end= {subject_end},s.subjectUri= '{subject_uri}'"
            
            session.run(subject_query)
            
            # Create the Object node if it doesn't exist
            object_query = f"MERGE (o:Object {{objectUri: '{object_uri}',start: {object_start}, end: {object_end}}}) \
                            ON CREATE SET o.objectText = '{object_text}',o.start= {object_start}, o.end= {object_end},o.objectUri= '{object_uri}'"
                            
            session.run(object_query)
            
            # Create the Fact-Property, Fact-Subject, and Fact-Object relationships
            '''
                        fact_property_query = f"MATCH (f:Fact {{factId: '{fact_id}'}}), \
                                        (pr:Property {{propertyId: '{property_id}'}}) \
                                    CREATE (f)-[:HAS_PROPERTY]->(pr)"
            session.run(fact_property_query)
            '''

            
            fact_subject_query = f"MATCH (f:Fact {{factId: '{fact_id}'}}), \
                    (s:Subject {{subjectUri: '{subject_uri}',start: {subject_start}, end: {subject_end}}}) \
                    CREATE (f)-[:HAS_SUBJECT {{start: {subject_start}, end: {subject_end}}}]->(s)"

            session.run(fact_subject_query)
            
            fact_object_query = f"MATCH (f:Fact {{factId: '{fact_id}'}}), \
                            (o:Object {{objectUri: '{object_uri}',start: {object_start}, end: {object_end}}}) \
                    CREATE (f)-[:HAS_OBJECT {{start: {object_start}, end: {object_end}}}]->(o)"

            session.run(fact_object_query)



In [75]:
from neo4j import GraphDatabase
import json
import uuid  

# Connect to Neo4j
uri = "bolt://localhost:7687"
username = "neo4j"
password = "B@bu12345"
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session(database="knwgrph")

# Load the JSON data
with open("json-fixer.json", "r") as file:
    data = json.load(file)

# Iterate through the folds
for fold_data in data:
    fold = fold_data['fold']
    print("In Fold loop{}".format(fold))
    source = fold_data['source']
    document_id = fold_data['documentId']
    document_text = fold_data['documentText'].replace("'", "\\'")

    # Create the Document node
    '''
        query = f"MERGE(d:Dcoumet{{CREATE (d:Document {{documentId: '{document_id}', documentText: '{document_text}'}})"
    session.run(query)
    '''

    
    query = f"MERGE (d:Document {{documentId: '{document_id}', documentText: '{document_text}', source: '{source}'}}) \
                            ON CREATE SET d.documentId = '{document_id}',d.documentText= '{document_text}', d.source= '{source}'"
    session.run(query)                    

    # Iterate through the passages
    for passage_data in fold_data['passages']:
        passage_id = passage_data['passageId']
        print("In Passage loop{}".format(passage_id))
        passage_start = passage_data['passageStart']
        passage_end = passage_data['passageEnd']
        passage_text = passage_data['passageText'].replace("'", "\\'")
        
        query = f"MERGE (p:Passage {{passageId: '{passage_id}', passageStart: '{passage_start}', passageEnd: '{passage_end}',passageText: '{passage_text}'}}) \
                            ON CREATE SET p.passageId = '{passage_id}',p.passageStart= {passage_start},p.passageEnd= {passage_end}, p.passageText= '{passage_text}'"
        session.run(query) 
        
        '''
                query = f"CREATE (p:Passage {{passageId: '{passage_id}', passageStart: {passage_start}, passageEnd: {passage_end}, passageText: '{passage_text}'}})"
        session.run(query)
        '''
        # Create the Passage node


        # Create the Document-Passage relationship
        query = f"MATCH (d:Document {{documentId: '{document_id}'}}), (p:Passage {{passageId: '{passage_id}'}}) CREATE (d)-[:HAS_PASSAGE]->(p)"
        session.run(query)

        # Iterate through the exhaustivelyAnnotatedPropertie            
        for property_data in passage_data['exhaustivelyAnnotatedProperties']:
            property_id = property_data['propertyId']
            property_name = property_data['propertyName']
            property_description = property_data['propertyDescription'].replace("'", "\\'")

            # Create the Property node
            prop_id = uuid.uuid1() 
            '''query = f"CREATE (pr:Property {{prop_id: '{prop_id}',propertyId: '{property_id}', propertyName: '{property_name}', propertyDescription: '{property_description}'}})"
            session.run(query)
            '''
            
            query = f"MERGE (pr:Property {{prop_id: '{prop_id}', propertyId: '{property_id}', propertyName: '{property_name}',propertyDescription: '{property_description}'}}) \
                            ON CREATE SET pr.prop_id = '{prop_id}',pr.propertyId= '{property_id}',pr.propertyName= '{property_name}', pr.propertyDescription= '{property_description}'"
            session.run(query) 

            
            # Create the Document-Passage relationship
            query = f"MATCH (p:Passage {{passageId: '{passage_id}'}}), (pr:Property {{prop_id: '{prop_id}'}}) CREATE (p)-[:HAS_PROPERTY]->(pr)"
            session.run(query)
                
        # Iterate through the facts
        for fact in passage_data['facts']:
            # Extract information from the fact
            fact_id = fact['factId']
            property_id = fact['propertyId']
            human_readable = fact['humanReadable']
            annotated_passage = fact['annotatedPassage']
            subject_start = fact['subjectStart']
            subject_end = fact['subjectEnd']
            subject_text = fact['subjectText'].replace("'", "\\'")
            subject_uri = fact['subjectUri']
            object_start = fact['objectStart']
            object_end = fact['objectEnd']
            object_text = fact['objectText'].replace("'", "\\'")
            object_uri = fact['objectUri']
            #property_name = fact['propertyName'].replace("'", "\\'")
            #property_desc = fact['propertyDescription'].replace("'", "\\'")
            humarn_readable = human_readable.replace("'", "\\'")
            annotated_passage = annotated_passage.replace("'", "\\'")
            
            '''
                        # Create the Property node if it doesn't exist
            property_query = f"MERGE (pr:Property {{propertyId: '{property_id}'}}) \
                            ON CREATE SET pr.propertyName = '{property_name}', \
                                            pr.propertyDescription = '{property_description}'"
            session.run(property_query)
            '''
            
            # Create the Fact node
            fact_query = f"MERGE (f:Fact {{factId: '{fact_id}'}}) \
                    ON CREATE SET f.humanReadable = '{humarn_readable}', \
                                f.annotatedPassage = '{annotated_passage}', f.propertyId = '{property_id}'"
            session.run(fact_query)
            
            query = f"MATCH (p:Passage {{passageId: '{passage_id}'}}), (f:Fact {{factId: '{fact_id}'}}) CREATE (p)-[:HAS_FACT]->(f)"
            session.run(query)
            
            # Create the Subject node if it doesn't exist
            subject_query = f"MERGE (s:Subject {{subjectUri: '{subject_uri}',start: {subject_start}, end: {subject_end}}}) \
                            ON CREATE SET s.subjectText = '{subject_text}',s.start= {subject_start}, s.end= {subject_end},s.subjectUri= '{subject_uri}'"
            
            session.run(subject_query)
            
            # Create the Object node if it doesn't exist
            object_query = f"MERGE (o:Object {{objectUri: '{object_uri}',start: {object_start}, end: {object_end}}}) \
                            ON CREATE SET o.objectText = '{object_text}',o.start= {object_start}, o.end= {object_end},o.objectUri= '{object_uri}'"
                            
            session.run(object_query)
            
            # Create the Fact-Property, Fact-Subject, and Fact-Object relationships
            '''
                        fact_property_query = f"MATCH (f:Fact {{factId: '{fact_id}'}}), \
                                        (pr:Property {{propertyId: '{property_id}'}}) \
                                    CREATE (f)-[:HAS_PROPERTY]->(pr)"
            session.run(fact_property_query)
            '''

            
            fact_subject_query = f"MATCH (f:Fact {{factId: '{fact_id}'}}), \
                    (s:Subject {{subjectUri: '{subject_uri}',start: {subject_start}, end: {subject_end}}}) \
                    CREATE (f)-[:HAS_SUBJECT {{start: {subject_start}, end: {subject_end}}}]->(s)"

            session.run(fact_subject_query)
            
            fact_object_query = f"MATCH (f:Fact {{factId: '{fact_id}'}}), \
                            (o:Object {{objectUri: '{object_uri}',start: {object_start}, end: {object_end}}}) \
                    CREATE (f)-[:HAS_OBJECT {{start: {object_start}, end: {object_end}}}]->(o)"

            session.run(fact_object_query)



In Fold loop3
In Passage loop13205:453:779
In Fold loop4
In Passage loop13115:1833:1922
In Fold loop2
In Passage loop7109:41:205
In Fold loop2
In Passage loop12774:25:153
In Fold loop3
In Passage loop10263:549:764
In Fold loop2
In Passage loop20474:976:1087
In Fold loop2
In Passage loop10629:111:220
In Fold loop2
In Passage loop5102:363:423
In Fold loop4
In Passage loop8275:16:86
In Fold loop2
In Passage loop36:13:155
In Fold loop3
In Passage loop28558:15:231
In Fold loop2
In Passage loop19392:23:133
In Fold loop3
In Passage loop7988:353:621
In Fold loop2
In Passage loop4966:25:140
In Passage loop4966:141:311
In Fold loop4
In Passage loop7791:1521:1693
In Fold loop1
In Passage loop9765:261:477
In Fold loop4
In Passage loop5051:199:307
In Fold loop3
In Passage loop19160:399:544
In Passage loop19160:309:398
In Fold loop1
In Passage loop7978:818:995
In Fold loop4
In Passage loop28361:20:205
In Fold loop3
In Passage loop1743:450:549
In Fold loop2
In Passage loop2278:17:146
In Fold loop2
In

In [134]:
subject_name = "Rich Brightman"
properrty_name = "DATE_OF_BIRTH"

query = f"MATCH (p:Passage)-[:HAS_FACT]-(f:Fact),(f:Fact)-[:HAS_SUBJECT]-(s:Subject),(f:Fact)-[:HAS_OBJECT]-(o:Object), (p:Passage)-[:HAS_PROPERTY]-(pr:Property) \
WHERE pr.propertyName='{properrty_name}' and s.subjectText='{subject_name}'\
RETURN o.objectText as DATE_OF_BIRTH"

info = session.run(query)   

print(info.values()[0][0])

April 17, 1990


In [141]:
subject_name = "Heraclio"
properrty_name = "NATIONALITY"

query1 = f"MATCH (p:Passage)-[:HAS_FACT]-(f:Fact),(f:Fact)-[:HAS_SUBJECT]-(s:Subject),(f:Fact)-[:HAS_OBJECT]-(o:Object), (p:Passage)-[:HAS_PROPERTY]-(pr:Property) \
WHERE pr.propertyName='{properrty_name}' and s.subjectText='{subject_name}' and f.humanReadable contains '{properrty_name}' \
RETURN o.objectText"


query2 = f"MATCH(d:Document)-[:HAS_PASSAGE]-(p:Passage),(p:Passage)-[:HAS_FACT]-(f:Fact),(f:Fact)-[:HAS_SUBJECT]-(s:Subject) \
WHERE s.subjectText='{subject_name}' \
RETURN d.documentId limit 1"
doc_id =""


info = session.run(query1)   
list_values = info.values()
#print(type(list_values))
#print(len(list_values))
if len(list_values) >0:
    for item in list_values:
        print(item[0])
if len(list_values) ==0:
    #print("I am in if")
    docs = session.run(query2)
    for items in docs:
        doc_id = items.values()[0]
        #print(doc_id)
        query3 = f"MATCH(d:Document)-[:HAS_PASSAGE]-(p:Passage),(p:Passage)-[:HAS_PROPERTY]-(pr:Property),(p:Passage)-[:HAS_FACT]-(f:Fact),(f:Fact)-[:HAS_SUBJECT]-(s:Subject),(f:Fact)-[:HAS_OBJECT]-(o:Object)\
            WHERE (s.subjectText='He' or s.subjectText='She' or s.subjectText='Who') and d.documentId= '{doc_id}' \
            and pr.propertyName = '{properrty_name}' and f.humanReadable contains '{properrty_name}' \
            RETURN o.objectText"
        mila = session.run(query3)  
        for item in mila:
            print(item.values()[0])
        
